In [ ]:
from playwright.async_api import async_playwright
import asyncio
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re

In [ ]:
urls = {
  'https://www.raiplay.it/genere/Fiction---Tutti-97cca670-a42a-498c-97fd-4fdb6b7f3398.html' : 'Fiction',
  'https://www.raiplay.it/genere/Film---Tutti-377b40d5-e723-4bfa-bc64-3e44c0d5b6e3.html' : 'Film',
  'https://www.raiplay.it/genere/Documentari---Tutti-a0e22b91-cb25-4789-8e82-9a976ce29138.html' : 'Documentari',
  'https://www.raiplay.it/raccolta/Antologie-Teche-in-esclusiva-14bc0b54-1dd1-4b5c-97b0-68775aa8e79c.html' : 'Antologie',
  'https://www.raiplay.it/genere/Teche-Rai---Tutti-e91c17fd-b21d-4ed3-bb14-1fec5a89f5ad.html' : 'Teche RAI',
  'https://www.raiplay.it/genere/Serie-TV---Tutti-e0b70800-f090-4e49-8fcb-c28094afbe79.html': 'Serie internazionali',
  'https://www.raiplay.it/genere/Musica---Tutti-9f034f64-afb6-409f-ba18-f61182c3270f.html' : 'Musica',
  'https://www.raiplay.it/genere/Film-per-ragazzi-6e567ae1-4d10-4894-8002-00f79fea4b21.html' : 'Film per ragazzi',
  'https://www.raiplay.it/genere/Bambini---Tutti-eda7ecb7-8ff6-419e-9c03-87d02e3fa454.html' : 'Bambini',
  'https://www.raiplay.it/genere/Programmi---Tutti-20269973-8d0d-4cc4-9f82-66bd9fa2b03a.html': 'Programmi'
}

In [ ]:
# List to store movie names and genres
movies = []

async def scrape_movies(url, genre, page):
    # Go to the URL
    await page.goto(url)
    print(f"Going to {url}")
    
    # Get the HTML content of the page
    html_content = await page.content()
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all card items with movie names
    cards = soup.find_all('a', class_='card-item__link')
    
    for card in cards:
        movie_name = card['aria-label'].replace('Maggiori informazioni su ', '')
        movies.append((movie_name, genre))
    await page.wait_for_timeout(2000)

In [ ]:
# Start Playwright
playwright = await async_playwright().start()

# Launch the browser
browser = await playwright.chromium.launch(headless=False)

# Create a new browser page
page = await browser.new_page()

# Loop through URLs and scrape movies
for url, genre in urls.items():
    await scrape_movies(url, genre, page)

# Close the browser
await browser.close()

# Print the scraped movie names and genres
for movie, genre in movies:
    print(f"Movie: {movie}, Genre: {genre}")

In [ ]:
df = pd.DataFrame(movies, columns=['Movie Name', 'Genre'])
df.head()


In [ ]:
df['Genre'].value_counts()

In [ ]:
df.to_csv('catalogo.csv', index=False, encoding='UTF-8-sig')